In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Trabajo Práctico 1
Se buscó analizar si existe un sesgo en el dataset de snli donde el contenido mismo del texto puede estar delatando si existe una contradicción con respecto a la hipótesis oculta.
## Carga de Datos
Se cargaron los datasets correspondientes para observación, ya divididos en datos de entrenamiento, validación y prueba.

In [4]:
# Cargo los datos
df_train = pd.read_hdf("/kaggle/input/sesgos-en-el-dataset-de-snli/train_data.hdf5")
df_valid = pd.read_hdf("/kaggle/input/sesgos-en-el-dataset-de-snli/valid_data.hdf5")
df_test = pd.read_hdf("/kaggle/input/sesgos-en-el-dataset-de-snli/test_data.hdf5")

In [5]:
df_submission = pd.read_csv("/kaggle/input/sesgos-en-el-dataset-de-snli/submission_sample.csv", index_col="pairID")

In [6]:
text_train = df_train["text"].tolist()
labels_train = df_train["gold_label"].tolist()
text_val = df_valid["text"].tolist()
labels_val = df_valid["gold_label"].tolist()
text_test = df_test["text"].tolist()

In [7]:
#Veamos el balance de clases
from collections import Counter
Counter(labels_train)

## Pre-procesamiento de Texto
+ NLTK (Natural Language Toolkit)
  + Lemmatization: reduce a sus significados (ej, quita conjugación verbal)
  + Stop Words: quita preposiciones (como palabras muy usuales de relleno?)
  + Stemming: reduce las palabras a su raíz
  + Filtrado de no palabras

In [8]:
# Paquetes de Natural Language Tool Kit
import nltk
#Tokenización (a partir de este se trabajan las otras combinacionies)
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [9]:
# Función con la cual también decido cómo pre-procesar
def text_filter(dataset, do_lemm, do_stop, do_stem, do_alpha):
    texts_filtrados = list()
    for idx in range(len(dataset.text)):
        if idx%100==0:
            print("\r Procesados: {}".format(idx),end="")
        em = dataset.text[idx]
        tok = word_tokenize(em)
        if do_lemm == True:
            lem = [lemmatizer.lemmatize(x,pos='v') for x in tok]
        else:
            lem = tok
        if do_stop == True:
            stop = [x for x in lem if x not in stopwords.words('english')]
        else:
            stop = lem
        if do_stem == True:
            stem = [stemmer.stem(x) for x in stop]
        else:
            stem = stop
        if do_alpha == True:
            alpha = [x for x in stem if x.isalpha()]
        else:
            alpha = stem
        texts_filtrados.append(" ".join(alpha))
    return texts_filtrados

## Vectorizadores
+ Count Vectorizer
+ TFIDF Vectorizer

In [10]:
#Importo los vectorizadores
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [11]:
df_max = 1.0 # max_df: int para frecuencia contada, float para proporcional
df_min = 10 # min_df: idem
n_range = (1,1) # ngram_range: (1,1) default
cv_cv = CountVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)

In [12]:
df_max = 1.0
df_min = 10
n_range = (1,1)
cv_idf = TfidfVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)

In [13]:
def get_cvs(text_train, text_valid, cv):
    cv_train = cv.fit_transform(text_train)
    cv_valid = cv.transform(text_valid)
    return cv_train, cv_valid

## Calisificadores
+ Multinomial Naive-Bayes
+ Regresión Logística (MLP)

### Multinomial Naive-Bayes

In [14]:
from sklearn.naive_bayes import MultinomialNB

In [15]:
# Parámetros del Clasificador
a = 1e-10
clf_NBMN = MultinomialNB(alpha = a)

### Multilevel Perceptrons

In [18]:
import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(hidden_units, input_shape=(input_dim,), activation='relu'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# Dimm capa entrada: num de vocablos
# Dimm capa salida: 3

## Métricas
+ Primaria:
 + asdf
+ Secundarias:
 + Precision
 + Recall
 + F1-score
 + ROC-AUC

In [22]:
from sklearn import metrics

In [23]:
def get_scores(clf, X_train, y_train, X_valid, y_valid):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    
    score_train = clf.score(X_train, y_train)
    score_valid = clf.score(X_valid, y_valid)
    return (score_train, score_valid)
    
def get_metrics(clf, X_valid, y_valid):
    y_pred = clf.predict(X_test)
    m_conf = metrics.confusion_matrix(y_valid, y_pred)
    precision = metrics.precision_score(y_valid, y_pred)
    recall_score = metrics.recall_score(y_valid,y_pred)
    f1_score = metrics.f1_score(y_valid,y_pred)
    acc = metrics.accuracy_score(y_valid, y_pred)
    
    return score_train, score_valid, m_conf, precision, recall_score, f1_score, acc

In [ ]:
# Función que hace todo lo anterior de una
def process_data(clf, cv, df_train, df_valid, do_lemm, do_stop, do_stem, do_alpha):
    print("\n  Filtrando Textos")
    texts_train = text_filter(df_train, do_lemm, do_stop, do_stem, do_alpha)
    texts_valid = text_filter(df_valid, do_lemm, do_stop, do_stem, do_alpha)
    
    labels_train = df_train["gold_label"].tolist()
    labels_valid = df_valid["gold_label"].tolist()
    
    print("\nVectorizando")
    cv_train, cv_valid = get_cvs(texts_train, texts_valid, cv)
    
    print("Obteniendo Puntajes")
    score_train, score_valid = get_scores(clf, cv_train, labels_train, cv_valid, labels_valid)
    return (cv_train, cv_valid, score_train, score_valid)

In [18]:
test_raw = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, False, False, False)
test_lemm = process_data(clf_NBMN, cv_cv, df_train, df_valid, True, False, False, False)
test_stop = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, True, False, False)
test_stem = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, False, True, False)
test_alfa = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, False, False, True)

In [19]:
print("Train/Valid:")
print("raw:  ",  test_raw[2],"; ",  test_raw[3])
print("lemm: ", test_lemm[2],"; ", test_lemm[3])
print("stop: ", test_stop[2],"; ", test_stop[3])
print("stem: ", test_stem[2],"; ", test_stem[3])
print("alfa: ", test_alfa[2],"; ", test_alfa[3])

Observando los resultados de estas pruebas, parece ser que aplicar cualquiera de los filtros de lenguaje natural resulta en una disminusión en la capacidad del modelo de predecir correctamente los resultados deseados. Esto puede deberse a que existe información dentro de cada elemento eliminado que podría llevar a concluir si la frase es contradictoria, neutral o afirmativa.

Debido a esto, se procederá a trabajar con los datos sin aplicar los filtros del paquete NLTK.

In [ ]:
# Finalmente se trabaja sin filtros
text_train_final = text_filter(df_train, True, False, True, False)
text_valid_final = text_filter(df_valid, True, False, True, False)

In [20]:
# Finalmente se trabaja sin filtros
text_train_final = text_train
text_valid_final = text_val

Definido el pre-procesamiento se analizó si se obtienen mejores resultados utilizando el CountVectorizer o TFIDFVectorizer.

In [34]:
df_max = 1.0 # max_df: int para frecuencia contada, float para proporcional
df_min = 10 # min_df: idem
n_range = (1,2) # ngram_range: (1,1) default

In [35]:
cv_cv = CountVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)
cv_idf = TfidfVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)

In [36]:
cv_train, cv_valid = get_cvs(text_train_final, text_valid_final, cv_cv)
cv_scores = get_scores(clf_NBMN, cv_train, labels_train, cv_valid, labels_val)
cv_train, cv_valid = get_cvs(text_train_final, text_valid_final, cv_idf)
idf_scores = get_scores(clf_NBMN, cv_train, labels_train, cv_valid, labels_val)

print(cv_scores)
print(idf_scores)

Bajo las mismas condiciones el TFIDF Vectorizer parece tener un mejor rendimiento a la hora de predecir los resultados correctamente. Por lo tanto las siguientes pruebas se harán con el TFIDF.



In [57]:
# Hago un Sweep para optimizar el min_df
train_scores = list()
valid_scores = list()
df_mins = range(5,15,1)
for i in df_mins:
    cv = TfidfVectorizer(max_df = df_max, min_df= i, ngram_range = n_range)
    cv_train, cv_valid = get_cvs(text_train_final, text_valid_final, cv)
    scores = get_scores(clf_NBMN, cv_train, labels_train, cv_valid, labels_val)
    train_scores.append(scores[0])
    valid_scores.append(scores[1])

In [58]:
import matplotlib.pyplot as plt
plt.plot(df_mins,train_scores)
plt.plot(df_mins,valid_scores)

Con un min_df = 8 se obtuvo el score_valid más alto.

In [ ]:
# Puedo hacer el sweep para ver qué onda si cambio max_df

Algo pasa más arriba de acá

In [ ]:
clf.fit(cv_train, labels_train)
labels_pred = clf.predict(cv_valid)

In [ ]:
#Veamos cómo funciona el clasificador para train
clf.score(cv_train, labels_train)

In [ ]:
# Veamos cómo funciona el clasificador para valid
clf.score(cv_valid, labels_val)

In [ ]:
cv_test = cv.transform(text_test_final)
get_metrics(clf, cv_test, labels_test)
# test_labels = clf.predict(cv_test)

## Modelo Final

In [60]:
# Parámetros de Pre-procesamiento (sin procesamiento)
text_train_final = text_train
text_valid_final = text_val

# Parámetros del Count Vectorizer
df_max = 1.0 # max_df: int para frecuencia contada, float para proporcional
df_min = 8 # min_df: obtenido de una curva
n_range = (1,2) # ngram_range: (1,1) default

cv = TfidfVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)

# Parámetros del Clasificador
a = 1e-4
clf = MultinomialNB(alpha = a)

cv_train = cv.fit_transform(text_train_final)
clf.fit(cv_train, labels_train)

cv_test = cv.transform(text_test)
test_labels = clf.predict(cv_test)

## Métricas del modelo final

# Para armar Submission
Una vez elegido el grado de pre-procesamiento, el vectorizador y el clasificador, lo aplico sobre el el test.
+ Preprocesamiento: Lemmatization, Stemming y No-Palabras
+ Vectorizador: TFIDF
+ Clasificador: Multinomial Naive-Bayes

In [61]:
#Armo el submission.csv
df_test = pd.DataFrame(data=test_labels, columns=["pred_labels"],)

In [62]:
df_test.head()

In [63]:
df_test.index.names = ["pairID"]

In [64]:
df_test

In [65]:
df_test.to_csv("submission.csv")